In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../')

In [3]:
from fangraphsdownload import updatefangraphs

In [10]:
#updatefangraphs(link='https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=atc',path='hitters')
updatefangraphs(link='https://www.fangraphs.com/projections.aspx?pos=all&stats=pit&type=atc&team=0&lg=all&players=0',path='pitchers')
#updatefangraphs(link='https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=0&season=2022&month=0&season1=2022&ind=0&team=0&rost=0&age=0&filter=&players=0',path='positions')
#updatefangraphs(link='https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=0&season=2021&month=0&season1=2021&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=',path='positionsold', debug=True)


In [12]:
def GetPosition (array):
    if isinstance(array,np.ndarray):
        if array.__contains__('C'):
            return 'C'
        if array.__contains__('2B'):
            return '2B'
        if array.__contains__('SS'):
            return 'SS'
        if array.__contains__('3B'):
            return '3B'
        if array.__contains__('1B'):
            return '1B'
        if array.__contains__('OF'):
            return 'OF'
        else:
            return 'Util'
    else: 
        return 'Util'

In [15]:
atcbat = pd.read_csv("hitters/FanGraphs Leaderboard.csv")
atcpit = pd.read_csv("pitchers/FanGraphs Leaderboard.csv")

In [16]:
hittersmean = atcbat
pitchersmean = atcpit

In [ ]:
hittersmean

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,wRC+,-1.1,WAR,-1.2,ADP,-1.3,InterSD,InterSK,IntraSD,playerid
0,Juan Soto,WSN,153,661,519,162,30,3,36,112,...,174,NaN,7.6,NaN,3.9,NaN,2.47,0.74,0.98,20123
1,Mike Trout,LAA,136,587,475,134,25,3,37,99,...,164,NaN,6.6,NaN,12.9,NaN,3.23,-1.19,0.74,10155
2,José Ramírez,CLE,148,633,551,149,35,4,35,100,...,140,NaN,5.9,NaN,3.2,NaN,2.94,0.20,0.85,13510
3,Mookie Betts,LAD,147,657,565,158,34,3,31,112,...,138,NaN,5.8,NaN,15.5,NaN,3.44,-0.21,0.65,13611
4,Vladimir Guerrero Jr.,TOR,152,650,568,174,32,2,41,107,...,159,NaN,5.6,NaN,5.3,NaN,4.17,0.04,1.44,19611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,Scott Schebler,NaN,11,33,30,6,1,0,1,3,...,43,NaN,-0.2,NaN,999.0,NaN,5.95,2.30,1.81,12225
621,Alex Blandino,NaN,27,72,60,13,3,0,0,8,...,71,NaN,-0.2,NaN,999.0,NaN,6.76,0.59,1.67,16271
622,Josh Reddick,NaN,45,155,142,35,7,1,3,16,...,75,NaN,-0.2,NaN,999.0,NaN,0.78,3.25,1.41,3892
623,Albert Pujols,NaN,47,172,161,38,6,0,8,17,...,84,NaN,-0.4,NaN,598.2,NaN,3.13,0.09,1.26,1177


In [17]:
hittersmean['OBPOnTeam'] = (hittersmean['PA']*hittersmean['OBP']+2037.4)/(hittersmean['PA']+6100)
hittersmean['SLGOnTeam'] = (hittersmean['AB']*hittersmean['SLG']+2420)/(hittersmean['AB']+5500)
hittersmean['OPSValue'] = (hittersmean['OBPOnTeam']+hittersmean['SLGOnTeam']-.774)/.006

hittersmean['AVGValue'] = ((hittersmean['H']+1768)/(hittersmean['AB']+6100)-.267)/.0024
hittersmean['value'] = hittersmean['OPSValue']+(hittersmean['R']/8.92+hittersmean['HR']/7.5+hittersmean['RBI']/8.92+hittersmean['SB']/6.0)

In [18]:
#https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=1&season=2020&month=0&season1=2020&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=
positions = pd.read_csv("positionsold/FanGraphs Leaderboard.csv")

In [19]:
positions['Pos']=positions['Pos'].replace(['LF','CF','RF'],'OF')


In [ ]:
positions_unique=positions.loc[(positions['G']>=10) | (positions['GS']>=5)].groupby('playerid')['Pos'].unique()


In [ ]:
positions_unique.index=positions_unique.index.map(str)

In [ ]:
hitterspos=pd.merge(hittersmean,positions_unique,how='left',left_on='playerid',right_index=True)


In [ ]:
hitterspos

,Name,Team,G,PA,AB,H,2B,3B,HR,R,...,InterSD,InterSK,IntraSD,playerid,OBPOnTeam,SLGOnTeam,OPSValue,AVGValue,value,Pos
0,Juan Soto,WSN,153,661,519,162,30,3,36,112,...,2.47,0.74,0.98,20123,0.345732,0.452675,4.067892,10.243680,35.028580,[OF]
1,Mike Trout,LAA,136,587,475,134,25,3,37,99,...,3.23,-1.19,0.74,10155,0.340671,0.450971,2.940361,9.282319,30.677132,[OF]
2,José Ramírez,CLE,148,633,551,149,35,4,35,100,...,2.94,0.20,0.85,13510,0.336256,0.448833,1.848184,8.844723,33.103042,[3B]
3,Mookie Betts,LAD,147,657,565,158,34,3,31,112,...,3.44,-0.21,0.65,13611,0.337792,0.446800,1.765427,9.155101,30.541511,"[2B, OF]"
4,Vladimir Guerrero Jr.,TOR,152,650,568,174,32,2,41,107,...,4.17,0.04,1.44,19611,0.339296,0.453573,3.144856,10.100730,33.214662,[1B]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,Scott Schebler,NaN,11,33,30,6,1,0,1,3,...,5.95,2.30,1.81,12225,0.333602,0.439458,-0.156778,9.331838,0.649201,NaN
621,Alex Blandino,NaN,27,72,60,13,3,0,0,8,...,6.76,0.59,1.67,16271,0.333802,0.438252,-0.324419,9.218074,1.299646,[1B]
622,Josh Reddick,NaN,45,155,142,35,7,1,3,16,...,0.78,3.25,1.41,3892,0.333034,0.438087,-0.479871,9.104053,3.507573,[OF]
623,Albert Pujols,NaN,47,172,161,38,6,0,8,17,...,3.13,0.09,1.26,1177,0.332519,0.439346,-0.355832,8.938468,5.473914,[1B]


In [ ]:
hitterspos['BestPos']=hitterspos['Pos'].apply(GetPosition)

hitterspos['rank']=hitterspos.groupby('BestPos')['value'].rank(ascending=False)
hitterspos['utilrank']=hitterspos['value'].rank(ascending=False)

###customized league replacement value
replacementvalue=hitterspos.groupby('BestPos').value.nlargest(10).groupby('BestPos').min().reset_index(name='PosValue')
replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==80]['value'])

replacementvalue['ReplacementValue']=pd.DataFrame([replacementvalue['UtilValue'], replacementvalue['PosValue']]).min()
#replacementvalue['ReplacementValue']=replacementvalue['UtilValue']
replacementvalue.loc[replacementvalue['BestPos'] == 'Util', 'ReplacementValue'] = replacementvalue['UtilValue']

In [ ]:
replacementvalue

,BestPos,PosValue,UtilValue,ReplacementValue
0,1B,23.768028,21.832814,21.832814
1,2B,24.004127,21.832814,21.832814
2,3B,21.411536,21.832814,21.411536
3,C,14.159074,21.832814,14.159074
4,OF,27.533485,21.832814,21.832814
5,SS,21.838785,21.832814,21.832814
6,Util,11.156581,21.832814,21.832814


In [ ]:
hittersfinal=hitterspos.reset_index().merge(replacementvalue,on='BestPos',how='left')

hittersfinal['VORP']=hittersfinal['value']-hittersfinal['ReplacementValue']


In [ ]:
hittersfinal

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,AVGValue,value,Pos,BestPos,rank,utilrank,PosValue,UtilValue,ReplacementValue,VORP
0,0,Juan Soto,WSN,153,661,519,162,30,3,36,...,10.243680,35.028580,[OF],OF,1.0,1.0,27.533485,21.832814,21.832814,13.195766
1,1,Mike Trout,LAA,136,587,475,134,25,3,37,...,9.282319,30.677132,[OF],OF,3.0,8.0,27.533485,21.832814,21.832814,8.844318
2,2,José Ramírez,CLE,148,633,551,149,35,4,35,...,8.844723,33.103042,[3B],3B,1.0,3.0,21.411536,21.832814,21.411536,11.691506
3,3,Mookie Betts,LAD,147,657,565,158,34,3,31,...,9.155101,30.541511,"[2B, OF]",2B,2.0,9.0,24.004127,21.832814,21.832814,8.708697
4,4,Vladimir Guerrero Jr.,TOR,152,650,568,174,32,2,41,...,10.100730,33.214662,[1B],1B,1.0,2.0,23.768028,21.832814,21.832814,11.381848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,620,Scott Schebler,NaN,11,33,30,6,1,0,1,...,9.331838,0.649201,NaN,Util,96.0,603.0,11.156581,21.832814,21.832814,-21.183613
621,621,Alex Blandino,NaN,27,72,60,13,3,0,0,...,9.218074,1.299646,[1B],1B,59.0,540.0,23.768028,21.832814,21.832814,-20.533167
622,622,Josh Reddick,NaN,45,155,142,35,7,1,3,...,9.104053,3.507573,[OF],OF,135.0,427.0,27.533485,21.832814,21.832814,-18.325240
623,623,Albert Pujols,NaN,47,172,161,38,6,0,8,...,8.938468,5.473914,[1B],1B,50.0,377.0,23.768028,21.832814,21.832814,-16.358900


In [ ]:

########################
#CHANGE THE COLUMNS BELOW BASED ON SGP VALUES FOR YOUR LEAGUE
pitchersmean['ERA']=((525.55+pitchersmean['ER'])*9/(1350.0+pitchersmean['IP'])-3.5)/-0.1062
pitchersmean['WHIP'] =((1620.0+pitchersmean['H']+pitchersmean['BB'])/(1350.0+pitchersmean['IP'])-1.2)/-0.0201
pitchersmean['value'] = pitchersmean['ERA']+pitchersmean['WHIP']+(pitchersmean['W']/4.05+pitchersmean['K/9']*pitchersmean['IP']/(55.0*9.0)+(pitchersmean['SV']+pitchersmean['HLD'])/10.0)
########################

pitchersmean['utilrank']=pitchersmean['value'].rank(ascending=False)

###customized league replacement value
pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==100]['value'])

pitchersmean['VORP']=pitchersmean['value']-pitchersmean['ReplacementValue']


In [ ]:
pitchersmean

,Name,Team,GS,G,IP,W,L,QS,SV,HLD,...,ADP,-1.2,InterSD,InterSK,IntraSD,playerid,value,utilrank,ReplacementValue,VORP
0,Corbin Burnes,MIL,29,29,171.0,12,7,16,0,0,...,10.6,NaN,3.96,-0.20,1.40,19361,8.280541,2.0,4.127139,4.153402
1,Jacob deGrom,NYM,24,24,145.0,10,5,16,0,0,...,17.8,NaN,5.73,-0.03,1.93,10954,8.263672,3.0,4.127139,4.136533
2,Gerrit Cole,NYY,30,30,189.0,16,8,18,0,0,...,6.9,NaN,4.32,0.35,1.57,13125,9.925830,1.0,4.127139,5.798691
3,Shane Bieber,CLE,28,28,174.0,13,8,17,0,0,...,31.2,NaN,7.45,-1.70,1.08,19427,7.905764,5.0,4.127139,3.778625
4,Zack Wheeler,PHI,28,28,179.0,12,8,16,0,0,...,32.6,NaN,4.22,0.47,0.84,10310,6.596440,15.0,4.127139,2.469301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,Yusmeiro Petit,NaN,0,66,66.0,4,4,0,0,10,...,601.0,NaN,3.63,0.37,0.56,4020,1.999501,297.0,4.127139,-2.127638
813,Pedro Báez,HOU,0,51,52.0,3,2,0,1,10,...,999.0,NaN,2.57,0.15,0.56,5420,2.065497,286.0,4.127139,-2.061642
814,Josh Tomlin,NaN,1,46,50.0,2,3,1,0,3,...,999.0,NaN,0.84,-1.30,0.60,9388,0.644649,532.0,4.127139,-3.482490
815,Jhoulys Chacín,COL,1,53,63.0,3,3,1,0,11,...,999.0,NaN,1.74,0.11,0.52,2608,1.340563,390.0,4.127139,-2.786576


In [ ]:
final=pd.concat([hittersfinal,pitchersmean])


TypeError: 'int' object is not callable

In [ ]:
final['VORP']=final['VORP'].round(2)

In [ ]:
final.sort_values('VORP', ascending=False)

,index,Name,Team,G,PA,AB,H,2B,3B,HR,...,QS,SV,HLD,ER,WHIP,K/9,BB/9,ERA,FIP,RA9-WAR
0,0.0,Juan Soto,WSN,153,661.0,519.0,162,30.0,3.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,José Ramírez,CLE,148,633.0,551.0,149,35.0,4.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,Vladimir Guerrero Jr.,TOR,152,650.0,568.0,174,32.0,2.0,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,Bryce Harper,PHI,150,646.0,528.0,146,32.0,2.0,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,37.0,Salvador Perez,KCR,139,587.0,547.0,144,25.0,1.0,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,529.0,Greg Deichmann,NaN,9,28.0,25.0,5,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,582.0,Taylor Davis,NaN,9,29.0,26.0,6,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,562.0,Nick Martini,NaN,8,20.0,17.0,4,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,603.0,Yusniel Diaz,BAL,6,25.0,23.0,5,1.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final.sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")

In [ ]:
final[['Name','Team','Pos','BestPos','G','GS','W','SV','VORP','ADP','rank','utilrank']].sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")